In [2]:
from bs4 import BeautifulSoup, Comment
import pandas as pd
import requests
import string
from time import sleep

In [7]:
df = pd.DataFrame()
site = 'https://www.hockey-reference.com'
prev_player=''
for letter in string.ascii_lowercase:
    count = 0
    url = requests.get(site+'/players/'+letter+'/').text
    soup = BeautifulSoup(url,'lxml')
    player_list = soup.find_all('p', class_='nhl')

    for count, player in enumerate(player_list):
        sleep(.1)
        if int(player.text.split('(')[-1][0:4])<1980 or player.text[-2] == 'G':
            continue
        if player.a.text == prev_player:
            if same_name_flag == True:
                same_name_count += 1
            same_name_flag = True
        else:
            same_name_flag = False
            same_name_count = 0
        player_name = player.a.text+same_name_flag*str(same_name_count)
        prev_player = player.a.text
        print(f'\r{count:03d}/{len(player_list):03d} of the letter {letter.upper()} completed, currently on: {player_name}                           ',end='')
        player_url = site+player.a['href']
        player_url = requests.get(player_url).text
        try:
            df_list = pd.read_html(player_url, skiprows = 1, header = 0)
        except:
            continue
        tmp_df = df_list[0]
        tmp_df = tmp_df[tmp_df.Lg == 'NHL']
        tmp_df.drop(columns = ['Lg','Awards'], inplace=True)
        if tmp_df['Age'].isnull().any():
            tmp_df.drop(tmp_df.loc[tmp_df['Age'].isnull()].index, inplace=True)
        tmp_df['Player'] = player_name
        if df.empty:
            df = tmp_df
        else:
            df = df.append(tmp_df, ignore_index=True)
print('\rCompleted!                                                              ',end='')

011/225 of the letter A completed, currently on: Thommy Abrahamsson                           

C:\Users\aronf\Anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\aronf\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Completed! the letter Z completed, currently on: Andrei Zyuzin                                           


In [14]:
df.to_csv('NHL_player_season_log_all.csv',index=False)

In [6]:
test_df = pd.read_html(requests.get('https://www.hockey-reference.com/players/g/gretzwa01.html').text, skiprows=1, header=0)
test_df[0][test_df[0].Lg=='NHL']

,Season,Age,Tm,Lg,GP,G,A,PTS,+/-,PIM,...,SH,GW,EV.1,PP.1,SH.1,S,S%,TOI,ATOI,Awards
3,1979-80,19.0,EDM,NHL,79,51,86,137,14,21,...,1.0,6.0,63.0,22.0,1.0,284,18.0,NaN,NaN,"AS-2,Byng-1,Hart-1"
4,1980-81,20.0,EDM,NHL,80,55,109,164,41,28,...,4.0,3.0,68.0,38.0,3.0,261,21.1,NaN,NaN,"AS-1,Byng-2,Hart-1,Ross-1"
5,1981-82,21.0,EDM,NHL,80,92,120,212,80,26,...,6.0,12.0,79.0,39.0,2.0,370,24.9,NaN,NaN,"AS-1,Byng-3,Hart-1,Pearson-1,Ross-1"
6,1982-83,22.0,EDM,NHL,80,71,125,196,61,59,...,6.0,9.0,85.0,36.0,4.0,348,20.4,NaN,NaN,"AS-1,Byng-5,Hart-1,Pearson-1,Ross-1"
7,1983-84,23.0,EDM,NHL,74,87,118,205,78,39,...,12.0,11.0,80.0,27.0,11.0,326,26.7,NaN,NaN,"AS-1,Byng-6,Hart-1,Pearson-1,Ross-1"
8,1984-85,24.0,EDM,NHL,80,73,135,208,100,52,...,11.0,7.0,92.0,36.0,7.0,354,20.6,NaN,NaN,"AS-1,Byng-15,Hart-1,Pearson-1,Ross-1,Selke-25,..."
9,1985-86,25.0,EDM,NHL,80,52,163,215,71,46,...,3.0,6.0,105.0,43.0,15.0,350,14.9,NaN,NaN,"AS-1,Byng-7,Hart-1,Ross-1,Selke-24"
10,1986-87,26.0,EDM,NHL,79,62,121,183,69,28,...,7.0,4.0,82.0,33.0,6.0,288,21.5,NaN,NaN,"AS-1,Byng-2,Hart-1,Pearson-1,Ross-1"
11,1987-88,27.0,EDM,NHL,64,40,109,149,39,24,...,5.0,3.0,65.0,40.0,4.0,211,19.0,NaN,NaN,"AS-2,Byng-2,Hart-3,Smythe-1"
12,1988-89,28.0,LAK,NHL,78,54,114,168,15,26,...,5.0,5.0,62.0,42.0,10.0,303,17.8,NaN,NaN,"AS-2,Byng-2,Hart-1"


In [155]:
year = 2020
site = 'https://www.hockey-reference.com'
year_url = site+'/leagues/NHL_'+str(year)+'.html'
url = requests.get(year_url).text
soup = BeautifulSoup(url,'lxml')

In [ ]:
r = requests.get(year_url)

if r.status_code == 200:
    soup = BeautifulSoup(r.content, 'html.parser')

    for comment in soup.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            table = comment_soup.find("table")

In [156]:
r = requests.get(year_url).text

In [157]:
soup = BeautifulSoup(r, 'html.parser')

In [170]:
site = 'https://www.hockey-reference.com'
stat_list = ['games','total_goals_per_game','goals_pp','chances_pp','shots','shot_pct']
stat_dict = {
    'year': [],
    'games': [],
    'total_goals_per_game': [],
    'goals_pp': [],
    'chances_pp': [],
    'shots': [],
    'shot_pct': []
}

for year in range(1977,2021):
    if year == 2005:
        continue # Stupid lockout
    sleep(0.1)
    print(f'\r{year}',end='')
    year_url = site+'/leagues/NHL_'+str(year)+'.html'
    url = requests.get(year_url).text
    
    soup = BeautifulSoup(url,'lxml')
    stat_dict['year'].append(f'{year-1}-{str(year)[-2:]}') 
    for comment in soup.find_all(text=lambda text: isinstance(text, Comment)):
        if comment.find("<table ") > 0:
            comment_soup = BeautifulSoup(comment, 'html.parser')
            table = comment_soup.find("table")
            table2=table.find('tfoot')
            for stat in stat_list:
                stat_dict[stat].append(table2.find('td',attrs={'data-stat':stat}).text)
            break
season_df = pd.DataFrame(stat_dict)

2020

In [171]:
season_df.to_csv('League_Average_Data.csv',index=False)

In [146]:
f'{year-1}-{str(year)[-2:]}'

'1976-77'

In [167]:
stat_dict

{'year': ['1976-77',
  '1977-78',
  '1978-79',
  '1979-80',
  '1980-81',
  '1981-82',
  '1982-83',
  '1983-84',
  '1984-85',
  '1985-86',
  '1986-87',
  '1987-88',
  '1988-89',
  '1989-90',
  '1990-91',
  '1991-92',
  '1992-93',
  '1993-94',
  '1994-95',
  '1995-96',
  '1996-97',
  '1997-98',
  '1998-99',
  '1999-00',
  '2000-01',
  '2001-02',
  '2002-03',
  '2003-04',
  '2005-06',
  '2006-07',
  '2007-08',
  '2008-09',
  '2009-10',
  '2010-11',
  '2011-12',
  '2012-13',
  '2013-14',
  '2014-15',
  '2015-16',
  '2016-17',
  '2017-18',
  '2018-19',
  '2019-20'],
 'games': ['80',
  '80',
  '80',
  '80',
  '80',
  '80',
  '80',
  '80',
  '80',
  '80',
  '80',
  '80',
  '80',
  '80',
  '80',
  '80',
  '84',
  '84',
  '48',
  '82',
  '82',
  '82',
  '82',
  '82',
  '82',
  '82',
  '82',
  '82',
  '82',
  '82',
  '82',
  '82',
  '82',
  '82',
  '82',
  '48',
  '82',
  '82',
  '82',
  '82',
  '82',
  '82',
  '70'],
 'total_goals_per_game': ['6.64',
  '6.59',
  '7.00',
  '7.03',
  '7.69',
  '8

In [168]:
test_df = pd.DataFrame(stat_dict)

In [169]:
test_df

,year,games,total_goals_per_game,goals_pp,chances_pp,shots,shot_pct
0,1976-77,80,6.64,53,265,2401,11.1
1,1977-78,80,6.59,54,255,2343,11.3
2,1978-79,80,7.00,62,271,2351,11.9
3,1979-80,80,7.03,61,280,2352,12.0
4,1980-81,80,7.69,77,340,2441,12.6
5,1981-82,80,8.03,73,320,2489,12.9
6,1982-83,80,7.73,71,310,2444,12.7
7,1983-84,80,7.89,74,336,2448,12.9
8,1984-85,80,7.77,71,321,2447,12.7
9,1985-86,80,7.94,82,370,2484,12.8


In [13]:
df[df.Player == "Vitaly Abramov"]

,Season,Age,Tm,GP,G,A,PTS,+/-,PIM,EV,...,ATOI,Player,TSA,FOW,FOL,FO%,BLK,HIT,TK,GV
34,2018-19,20.0,OTT,1,0,0,0,-3.0,0,0.0,...,13:52,Vitaly Abramov,5.0,0.0,0.0,NaN,1.0,0.0,4.0,2.0
35,2019-20,21.0,OTT,2,1,0,1,0.0,2,1.0,...,5:47,Vitaly Abramov,4.0,0.0,0.0,NaN,0.0,0.0,0.0,1.0
